In [1]:
headers = {
		"Content-Type": "application/json",
		"Connection": "keep-alive",
		"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:120.0) Gecko/20100101 Firefox/120.0",
		"Access-Control-Allow-Origin": "*"
}

import re

def remove_whitespace(s: str, keep_spaces: bool = True) -> str:
    """
    Clean whitespace while keeping natural spaces between words.

    - Convert all non-space whitespace (tabs, newlines, etc.) into a space
    - Collapse multiple spaces into one
    - Trim leading/trailing spaces
    """
    # Replace tabs/newlines/etc. with a space
    s = re.sub(r"[^\S ]+", " ", s)

    # Collapse multiple spaces into one
    s = re.sub(r" +", " ", s)
    
    # Replace \ by ""
    s = re.sub(r"\\", "", s)
    
    # Trim
    return s.strip()


def get_specific_element(regex: str, text: str):
    m = re.search(regex, text)
    return m.group(0) if m else None

def safe_text(element, keep_spaces=False):
    if not element:
        return ""
    return remove_whitespace(element, keep_spaces)




In [6]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import csv
import os

url = "https://www.paruvendu.fr/immobilier/vente/ile-de-france/?rechpv=1&tt=1&tbApp=1&tbDup=1&tbChb=1&tbLof=1&tbAtl=1&tbPla=1&tbMai=1&tbVil=1&tbCha=1&tbPro=1&tbHot=1&tbMou=1&tbFer=1&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&lol=0&ray=50"

base_url = "https://www.paruvendu.fr"
headers = {"User-Agent": "Mozilla/5.0"}

# Charge la derniere progression enregistré t'as capté ? Toi meme tu sais frerot
progress_file = "data/progress.txt"
last_page = 1

if os.path.exists(progress_file):
    with open(progress_file, "r") as f:
        last_page = int(f.read().strip())

print("🎯 On recommence depuis la :", last_page)

pages = [url] + [url + "&p=" + str(i) for i in range(2, 501)]
pages = pages[last_page - 1:]  # ON PASS SI C'EST DEJA SCRAPÉ MAN

with open("data/annonces_link.csv", "a", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)

    for idx, p in enumerate(pages, start=last_page):
        try:
            print(f"[{idx}/500] Fetching: {p}")
            response = requests.get(p, headers=headers, timeout=10)

            if response.status_code != 200:
                print(f"⚠️ Failed page {idx}, status: {response.status_code}")
                continue

            soup = BeautifulSoup(response.text, "html.parser")
            blocs = soup.find_all("div", class_="blocAnnonce")

            print(f"Found {len(blocs)} annonces")

            for bloc in blocs:
                a_tag = bloc.find("a", href=True)
                if a_tag:
                    writer.writerow([base_url + a_tag["href"]])

            # Ca enregistre mon reuf
            with open(progress_file, "w") as f:
                f.write(str(idx))

            sleep(2)

        except Exception as e:
            print("❌ Error:", e)
            continue

print("✅ Done.")


🎯 On recommence depuis la : 16
[16/500] Fetching: https://www.paruvendu.fr/immobilier/vente/ile-de-france/?rechpv=1&tt=1&tbApp=1&tbDup=1&tbChb=1&tbLof=1&tbAtl=1&tbPla=1&tbMai=1&tbVil=1&tbCha=1&tbPro=1&tbHot=1&tbMou=1&tbFer=1&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&lol=0&ray=50&p=16
Found 30 annonces
[17/500] Fetching: https://www.paruvendu.fr/immobilier/vente/ile-de-france/?rechpv=1&tt=1&tbApp=1&tbDup=1&tbChb=1&tbLof=1&tbAtl=1&tbPla=1&tbMai=1&tbVil=1&tbCha=1&tbPro=1&tbHot=1&tbMou=1&tbFer=1&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&lol=0&ray=50&p=17
Found 30 annonces
[18/500] Fetching: https://www.paruvendu.fr/immobilier/vente/ile-de-france/?rechpv=1&tt=1&tbApp=1&tbDup=1&tbChb=1&tbLof=1&tbAtl=1&tbPla=1&tbMai=1&tbVil=1&tbCha=1&tbPro=1&tbHot=1&tbMou=1&tbFer=1&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&lol=0&ray=50&p=18
Found 30 annonces
[19/500] Fetching: https://www.paruvendu.fr/immobilier/vente/ile-de-france/?rechpv=1&tt=1&tbApp=1&tbDup=1&tbChb=1&tbLof=1&tbAtl=1&tbPla=1&tbMai=1&tbVil=1&tb